# Smart Beta Portfolio Optimisation with cvxpy and S&P500

The objective is to design a portfolio that closely tracks an index, while also minimizing the portfolio variance.  If this portfolio can match the returns of the index with less volatility, then it has a higher risk-adjusted return (same return, lower volatility).

We want to both minimize the portfolio variance and also want to closely track a market cap weighted index.  In other words, we're trying to minimize the distance between the weights of our portfolio and the weights of the index.

$Minimize \left [ \sigma^2_p + \lambda \sqrt{\sum_{1}^{m}(weight_i - indexWeight_i)^2} \right  ]$ where $m$ is the number of stocks in the portfolio, and $\lambda$ is a scaling factor that you can choose.

Why are we doing this? One way that investors evaluate a fund is by how well it tracks its index. The fund is still expected to deviate from the index within a certain range in order to improve fund performance.  A way for a fund to track the performance of its benchmark is by keeping its asset weights similar to the weights of the index.  We’d expect that if the fund has the same stocks as the benchmark, and also the same weights for each stock as the benchmark, the fund would yield about the same returns as the benchmark. By minimizing a linear combination of both the portfolio risk and distance between portfolio and benchmark weights, we attempt to balance the desire to minimize portfolio variance with the goal of tracking the index.



## Import libraries

In [1]:
pip install yfinance

     |████████████████████████████████| 6.3 MB 10.5 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23918 sha256=aae4fed58f4efe9938b3b68a2dbb1292a49f68f8313332ae80afe43818783150
  Stored in directory: /root/.cache/pip/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [2]:
import numpy as np
import pandas as pd
import cvxpy as cvx
import yfinance as yf
from scipy import stats
import requests
import bs4 as bs
import matplotlib.pyplot as plt
plt.style.use("seaborn-whitegrid")

## Load stock data

Focus on large dollar volume stocks to ensure liquidity.

In [3]:
# get S&P500 tickers from Wikipedia
resp = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'id': 'constituents'})
tickers = []

for row in table.find_all('tr')[1:]:
		ticker = row.find('td').text.strip()
		if "." in ticker:
			ticker = ticker.replace('.','-')
		tickers.append(ticker)

In [4]:
df = yf.download(tickers=tickers, period="max", interval="1d", group_by="ticker", auto_adjust=True)
df

[*********************100%***********************]  505 of 505 completed


IDXX              ...        MSFT              
                  Open        High  ...       Close        Volume
Date                                ...                          
1962-01-02         NaN         NaN  ...         NaN           NaN
1962-01-03         NaN         NaN  ...         NaN           NaN
1962-01-04         NaN         NaN  ...         NaN           NaN
1962-01-05         NaN         NaN  ...         NaN           NaN
1962-01-08         NaN         NaN  ...         NaN           NaN
...                ...         ...  ...         ...           ...
2021-08-09  685.200012  688.770020  ...  288.329987  1.610910e+09
2021-08-10  688.140015  688.140015  ...  286.440002  1.860150e+09
2021-08-11  672.640015  676.289978  ...  286.950012  1.395590e+09
2021-08-12  663.049988  676.890015  ...         NaN           NaN
2021-08-13  671.460022  678.000000  ...  292.850006  1.690518e+07

[15021 rows x 2525 columns]

## Get close prices

In [5]:
sp_close = df.copy()
sp_close = sp_close.loc[:, sp_close.columns.get_level_values(1)=="Close"].fillna(method="ffill") # ffill to fill any missing values (hilidays) with previous day's value
sp_close.columns = sp_close.columns.get_level_values(0)
sp_close = sp_close.loc["2017-01-01":"2021-01-01", :].dropna(axis=1)
sp_close

,IDXX,DIS,DXCM,BAC,MS,ABBV,YUM,HIG,DFS,AEE,XLNX,RTX,EXPD,HSIC,BAX,MGM,EOG,O,BBWI,WBA,AIG,EQR,WMB,NFLX,ICE,TRMB,BLL,KR,SYY,TEL,LRCX,KHC,VIAC,VLO,SRE,DPZ,QRVO,ZION,BMY,TXN,...,COST,OKE,LW,RHI,INFO,IFF,GM,WAB,DLTR,CNC,MKC,NKE,BSX,CAT,PFE,WST,NWSA,NSC,PEAK,RF,PHM,TFC,CB,BKR,STZ,NCLH,SBAC,SPG,MAA,XYL,COF,UPS,MRK,CAH,GPS,HSY,WRK,ENPH,MHK,MSFT
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-01-02,117.269997,99.803192,59.700001,20.157303,37.617527,50.367065,58.514866,42.961357,65.127533,46.251152,56.282799,61.905411,49.995193,59.494118,42.212635,27.243227,92.865952,47.043903,43.945141,72.259674,57.520432,55.307716,23.862875,123.800003,53.335739,30.150000,36.117554,31.323124,49.154427,63.481361,98.017120,71.659409,58.393520,54.879532,87.243599,152.813751,52.730000,38.414635,50.774410,64.437431,...,143.163269,41.645962,35.530758,44.501652,34.887691,106.518906,30.163183,80.518272,77.180000,28.254999,43.372322,48.420685,21.629999,81.876144,25.716291,83.023521,10.742725,98.732742,23.517553,12.408108,17.324579,40.129211,119.980148,39.456711,143.505173,42.529999,101.863045,140.785156,82.692039,46.841343,80.367653,99.585983,49.020733,61.012619,18.770803,93.142754,43.552029,1.010000,199.679993,57.992920
2017-01-03,117.599998,101.584358,58.250000,20.549498,38.329803,50.198158,58.403992,43.051517,65.705727,46.180630,55.070808,62.588734,50.127346,61.149021,42.307835,27.857450,95.107231,47.052097,44.439060,72.434296,57.353107,54.680386,24.108095,127.489998,53.146683,30.270000,36.444717,30.606079,48.825954,63.188133,99.435478,71.831757,59.100262,56.566391,87.668373,151.969254,52.900002,38.539585,51.651928,64.976089,...,142.823486,42.618023,35.192829,45.395695,35.390163,105.922264,30.431576,81.488129,77.449997,29.155001,43.493149,49.516171,21.879999,82.979713,26.128004,82.279716,10.958330,97.746048,23.565033,12.468594,17.484821,40.095074,119.607819,39.493141,144.853043,42.790001,104.102318,144.707520,82.185349,46.964317,81.869263,99.968216,50.086590,63.140480,19.844145,93.584015,43.938057,1.050000,201.750000,58.403568
2017-01-04,115.949997,102.886719,60.720001,20.932581,38.837303,50.905972,58.616505,43.772797,66.798859,46.233524,54.669926,62.628239,50.174553,61.392159,42.602959,27.838552,95.180725,47.755936,44.946320,72.451767,58.101727,55.238953,24.031462,129.410004,54.082561,30.969999,36.998005,30.170404,49.272511,62.922394,99.491127,71.421410,60.339352,54.534134,88.422569,156.412415,54.509998,39.092945,51.721897,64.896614,...,142.850296,42.509209,34.714073,46.444839,35.666039,106.121124,32.111160,82.933243,79.449997,30.125000,43.367668,50.554516,22.080000,82.608917,26.357616,83.483521,11.127064,101.080688,24.087296,12.632770,17.899555,40.402313,119.435303,39.323105,147.885849,44.770000,104.250305,145.864456,82.936935,47.664280,83.186615,99.994278,50.069927,64.115395,20.452650,93.764130,44.598587,1.150000,205.880005,58.142242
2017-01-05,115.959999,102.829262,62.660000,20.686308,38.481163,51.292057,58.810539,43.592480,65.859306,46.462761,54.007980,62.882378,50.061268,61.062744,42.888577,27.753508,95.695099,49.040894,41.408825,72.495415,57.775852,55.986599,24.322659,131.809998,53.855690,30.490000,36.978756,30.143171,49.111752,62.161865,99.268623,71.011116,60.100719,54.365444,88.214508,156.201263,53.360001,38.459255,52.587719,64.393265,...,145.666916,42.676052,34.376129,44.337437,35.754707,104.954964,31.505117,81.352341,78.059998,30.075001,42.517227,50.544979,22.049999,82.105698,26.610977,83.219261,11.145811,100.760933,24.649122,12.468594,17.842997,39.881710,118.627045,39.080177,137.364716,44.549999,102.869240,146.117981,82.827156,47.229168,81.417854,100.046379,50.053268,63.157436,19.649757,93.160767,44.804459,1.120000,204.350006,58.142242
2017-01-06,118.360001,104.361458,63.230000,20.686308,39.042088,51.308136,59.521992,43.682644,65.100449,46.815414,55.052166,63.560081,50.155666,61.247059,43.336010,27.895252,96.641228,48.975426,40.868183,72.556534,58.841526,56.184254,24.843756,131.070007,54.205456,30.440001

## Get volume

In [6]:
sp_volume = df.copy()
sp_volume = sp_volume.loc[:, sp_volume.columns.get_level_values(1)=="Volume"].fillna(method="ffill") # ffill to fill any missing values (hilidays) with previous day's value
sp_volume.columns = sp_volume.columns.get_level_values(0)
sp_volume = sp_volume.loc["2017-01-01":"2021-01-01", :].dropna(axis=1)
sp_volume

,IDXX,DIS,DXCM,BAC,MS,ABBV,YUM,HIG,DFS,AEE,XLNX,RTX,EXPD,HSIC,BAX,MGM,EOG,O,BBWI,WBA,AIG,EQR,WMB,NFLX,ICE,TRMB,BLL,KR,SYY,TEL,LRCX,KHC,VIAC,VLO,SRE,DPZ,QRVO,ZION,BMY,TXN,...,COST,OKE,LW,RHI,INFO,IFF,GM,WAB,DLTR,CNC,MKC,NKE,BSX,CAT,PFE,WST,NWSA,NSC,PEAK,RF,PHM,TFC,CB,BKR,STZ,NCLH,SBAC,SPG,MAA,XYL,COF,UPS,MRK,CAH,GPS,HSY,WRK,ENPH,MHK,MSFT
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-01-02,535100.0,6983300.0,1339300.0,72605900.0,6943500.0,5999200.0,1887100.0,1314500.0,1817900.0,950700.0,2111700.0,3496595.0,670700.0,1809225.0,2206500.0,5461200.0,2178200.0,2024000.0,1774848.0,3343200.0,4497200.0,1542200.0,3982200.0,4455400.0,1844400.0,943000.0,1638400.0,3897000.0,2318500.0,1145800.0,897900.0,2051400.0,3811600.0,2552900.0,939800.0,491400.0,1223200.0,1938100.0,7008900.0,4311800.0,...,1472300.0,1271200.0,773000.0,603500.0,2636800.0,504500.0,7646100.0,522300.0,1688500.0,2493400.0,1144400.0,7384300.0,4270800.0,3471200.0,19885400.0,284100.0,1944200.0,1487300.0,3985100.0,17954600.0,2948500.0,3467500.0,1351400.0,1872700.0,1145200.0,856900.0,609900.0,2285100.0,847500.0,646200.0,2424800.0,1734000.0,11289700.0,1613100.0,4763100.0,732900.0,811200.0,600600.0,440300.0,25579900.0
2017-01-03,569500.0,8884300.0,1429300.0,99298100.0,8316900.0,9328200.0,4793400.0,4256700.0,2292400.0,1107200.0,4127000.0,4353224.0,1729300.0,2188665.0,2937400.0,9439200.0,4151200.0,1973300.0,3473991.0,4942700.0,7321900.0,3005100.0,5750600.0,9437900.0,2595000.0,1697700.0,1830600.0,9555700.0,3877000.0,1941400.0,1937200.0,3104600.0,3553800.0,4425000.0,1113200.0,797600.0,1841200.0,2896200.0,10061700.0,4847900.0,...,2099000.0,1812500.0,2007000.0,1340500.0,2792200.0,531300.0,10904900.0,973900.0,2587900.0,3988200.0,1026800.0,13295600.0,6822400.0,5054900.0,23391800.0,566300.0,2011500.0,1770100.0,3083500.0,15937800.0,3364100.0,5674200.0,1401100.0,2710300.0,1991400.0,1618500.0,1269300.0,2751200.0,1006100.0,1596700.0,3441100.0,2390700.0,12564200.0,4459400.0,9145600.0,686200.0,1026000.0,445900.0,492800.0,20694100.0
2017-01-04,20606500.0,8322000.0,1811600.0,76875100.0,7747400.0,8674800.0,2835200.0,2988400.0,2360500.0,1114300.0,3281700.0,4912235.0,1547600.0,1066665.0,2161500.0,6330900.0,2417900.0,1808600.0,3990933.0,4125500.0,4927500.0,1934000.0,4273900.0,7843600.0,3074900.0,1374900.0,2219800.0,9158500.0,2986700.0,2208900.0,1297400.0,2274900.0,5365500.0,7431000.0,1389100.0,679400.0,1503800.0,1907500.0,6703400.0,4507100.0,...,2163600.0,1721300.0,1712300.0,1677200.0,1651300.0,705900.0,23388500.0,1092100.0,3259700.0,4213400.0,1335800.0,13759500.0,6458600.0,3772100.0,22754000.0,611100.0,1256000.0,2705500.0,5343100.0,9759700.0,6343400.0,4539800.0,1324400.0,2122600.0,2972300.0,1854500.0,704600.0,1991700.0,890100.0,1703000.0,2630900.0,1830800.0,9863700.0,2461500.0,9885100.0,844200.0,1225200.0,903000.0,544100.0,21340000.0
2017-01-05,1312200.0,6303000.0,1641400.0,86826400.0,9883200.0,5359400.0,4040000.0,1613400.0,2328200.0,2256100.0,2198000.0,5315046.0,891100.0,740520.0,2345000.0,8587100.0,3084800.0,2276500.0,10058666.0,7401400.0,4107900.0,1693800.0,7443700.0,10185500.0,1636800.0,1415500.0,1965800.0,9937900.0,3629400.0,1491300.0,1634300.0,3037400.0,4186500.0,6450600.0,2008100.0,482100.0,1343400.0,2106000.0,8422300.0,4301100.0,...,2880900.0,1342700.0,1099600.0,2510900.0,1786700.0,1010200.0,15636700.0,540400.0,2832000.0,2882400.0,2789800.0,8682200.0,5990600.0,4698800.0,21083600.0,446700.0,2412000.0,3135300.0,4948900.0,12715500.0,5055500.0,3619500.0,1731400.0,2028500.0,10511800.0,933700.0,691400.0,2186400.0,897900.0,1846000.0,2223900.0,1856200.0,10581400.0,1921000.0,13312700.0,1581800.0,937100.0,308700.0,540500.0,24876000.0
2017-01-06,946400.0,9551300.0,889700.0,66281500.0,9152800.0,4722300.0,2863800.0,2930100.0,3137800.0,1883800.0,2381800.0,4166358.0,882800.0,1056465.0,2930900.0,6989500.0,2418300.0,1856800.0,5098296.0,3949700.0,5603700.0,1638000.0,5847100.0,10657900.0,1490100.0,949700.0,1001800.0,9289700.0,4776500.0,1178500.0,1203600.0,2450200.0,5275

## Index weights

S&P500 is weighted by dividing the free-float market capitalization of a company in the index by the total market capitalization of the index.

In [7]:
def generate_dollar_volume_weights(close, volume):
  """
    Generate dollar volume weights.

    Parameters
    ----------
    close : DataFrame
        Close price for each ticker and date
    volume : str
        Volume for each ticker and date

    Returns
    -------
    dollar_volume_weights : DataFrame
        The dollar volume weights for each ticker and date
  """

  assert close.index.equals(volume.index)
  assert close.columns.equals(volume.columns)
  
  dollar_volume_traded = close * volume
  dollar_volume_weights = dollar_volume_traded.divide(dollar_volume_traded.sum(axis=1), axis=0)

  return dollar_volume_weights

In [8]:
sp_weights = generate_dollar_volume_weights(sp_close, sp_volume)
sp_weights

,IDXX,DIS,DXCM,BAC,MS,ABBV,YUM,HIG,DFS,AEE,XLNX,RTX,EXPD,HSIC,BAX,MGM,EOG,O,BBWI,WBA,AIG,EQR,WMB,NFLX,ICE,TRMB,BLL,KR,SYY,TEL,LRCX,KHC,VIAC,VLO,SRE,DPZ,QRVO,ZION,BMY,TXN,...,COST,OKE,LW,RHI,INFO,IFF,GM,WAB,DLTR,CNC,MKC,NKE,BSX,CAT,PFE,WST,NWSA,NSC,PEAK,RF,PHM,TFC,CB,BKR,STZ,NCLH,SBAC,SPG,MAA,XYL,COF,UPS,MRK,CAH,GPS,HSY,WRK,ENPH,MHK,MSFT
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-01-02,0.000743,0.008251,0.000947,0.017327,0.003092,0.003577,0.001307,0.000669,0.001402,0.000521,0.001407,0.002563,0.000397,0.001274,0.001103,0.001761,0.002395,0.001127,0.000923,0.002860,0.003063,0.001010,0.001125,0.006530,0.001165,0.000337,0.000701,0.001445,0.001349,0.000861,0.001042,0.001740,0.002635,0.001659,0.000971,0.000889,0.000764,0.000881,0.004213,0.003289,...,0.002495,0.000627,0.000325,0.000318,0.001089,0.000636,0.002731,0.000498,0.001543,0.000834,0.000588,0.004233,0.001094,0.003365,0.006054,0.000279,0.000247,0.001739,0.001110,0.002638,0.000605,0.001647,0.001920,0.000875,0.001946,0.000431,0.000736,0.003809,0.000830,0.000358,0.002307,0.002044,0.006552,0.001165,0.001059,0.000808,0.000418,0.000007,0.001041,0.017563
2017-01-03,0.000589,0.007933,0.000732,0.017937,0.002802,0.004116,0.002461,0.001611,0.001324,0.000449,0.001998,0.002395,0.000762,0.001176,0.001092,0.002311,0.003470,0.000816,0.001357,0.003147,0.003691,0.001444,0.001219,0.010577,0.001212,0.000452,0.000586,0.002571,0.001664,0.001078,0.001693,0.001960,0.001846,0.002200,0.000858,0.001065,0.000856,0.000981,0.004568,0.002769,...,0.002635,0.000679,0.000621,0.000535,0.000869,0.000495,0.002917,0.000698,0.001762,0.001022,0.000393,0.005787,0.001312,0.003687,0.005372,0.000410,0.000194,0.001521,0.000639,0.001747,0.000517,0.002000,0.001473,0.000941,0.002536,0.000609,0.001162,0.003500,0.000727,0.000659,0.002476,0.002101,0.005532,0.002475,0.001595,0.000564,0.000396,0.000004,0.000874,0.010624
2017-01-04,0.022104,0.007921,0.001018,0.014887,0.002784,0.004085,0.001537,0.001210,0.001459,0.000477,0.001660,0.002846,0.000718,0.000606,0.000852,0.001630,0.002129,0.000799,0.001659,0.002765,0.002649,0.000988,0.000950,0.009390,0.001538,0.000394,0.000760,0.002556,0.001361,0.001286,0.001194,0.001503,0.002995,0.003749,0.001136,0.000983,0.000758,0.000690,0.003208,0.002706,...,0.002859,0.000677,0.000550,0.000721,0.000545,0.000693,0.006948,0.000838,0.002396,0.001174,0.000536,0.006435,0.001319,0.002883,0.005548,0.000472,0.000129,0.002530,0.001191,0.001141,0.001050,0.001697,0.001463,0.000772,0.004067,0.000768,0.000680,0.002688,0.000683,0.000751,0.002025,0.001694,0.004569,0.001460,0.001870,0.000732,0.000506,0.000010,0.001036,0.011479
2017-01-05,0.001406,0.005989,0.000950,0.016597,0.003514,0.002540,0.002195,0.000650,0.001417,0.000969,0.001097,0.003088,0.000412,0.000418,0.000929,0.002202,0.002728,0.001032,0.003849,0.004958,0.002193,0.000876,0.001673,0.012406,0.000815,0.000399,0.000672,0.002768,0.001647,0.000857,0.001499,0.001993,0.002325,0.003241,0.001637,0.000696,0.000662,0.000748,0.004093,0.002559,...,0.003878,0.000529,0.000349,0.001029,0.000590,0.000980,0.004552,0.000406,0.002043,0.000801,0.001096,0.004055,0.001221,0.003565,0.005184,0.000344,0.000248,0.002919,0.001127,0.001465,0.000834,0.001334,0.001898,0.000733,0.013343,0.000384,0.000657,0.002952,0.000687,0.000806,0.001673,0.001716,0.004894,0.001121,0.002417,0.001362,0.000388,0.000003,0.001021,0.013365
2017-01-06,0.001065,0.009480,0.000535,0.013039,0.003398,0.002304,0.001621,0.001217,0.001943,0.000839,0.001247,0.002518,0.000421,0.000615,0.001208,0.001854,0.002223,0.000865,0.001982,0.002725,0.003136,0.000875,0.001381,0.013285,0.000768,0.000275,0.000354,0.002655,0.002247,0.000701,0.001142,0.001650,0.002960,0.001921,0.000793,0.000818,0.000483,0.000791,0.005038,0.003266,...,0.003000,0.000884,0.000450,0.000674,0.000529,0.001011,0.003923,0.000307,0.002276,0.000734,0.001034,0.006121,0.001590,0.003143,0.004643,0.000370,0.000181,0.001949,0.001176,0.000975,0.000883,0.001288,0.001810,0.000944,0.005909,0.000393,0.000606,0.003170,0.000735

In [9]:
sp_weights.iloc[-1].nlargest(10)

TSLA     0.198418
AAPL     0.074251
AMZN     0.054631
MSFT     0.026280
FB       0.019977
NFLX     0.016539
NVDA     0.014229
AMD      0.012969
BA       0.012734
GOOGL    0.010473
Name: 2020-12-31 00:00:00, dtype: float64

## Returns

In [10]:
def generate_returns(prices):
  """
    Generate returns for ticker and date.

    Parameters
    ----------
    prices : DataFrame
        Price for each ticker and date

    Returns
    -------
    returns : Dataframe
        The returns for each ticker and date
  """

  returns = prices / prices.shift(1) - 1
  return returns

In [11]:
sp_price_returns = generate_returns(sp_close)
sp_price_returns

,IDXX,DIS,DXCM,BAC,MS,ABBV,YUM,HIG,DFS,AEE,XLNX,RTX,EXPD,HSIC,BAX,MGM,EOG,O,BBWI,WBA,AIG,EQR,WMB,NFLX,ICE,TRMB,BLL,KR,SYY,TEL,LRCX,KHC,VIAC,VLO,SRE,DPZ,QRVO,ZION,BMY,TXN,...,COST,OKE,LW,RHI,INFO,IFF,GM,WAB,DLTR,CNC,MKC,NKE,BSX,CAT,PFE,WST,NWSA,NSC,PEAK,RF,PHM,TFC,CB,BKR,STZ,NCLH,SBAC,SPG,MAA,XYL,COF,UPS,MRK,CAH,GPS,HSY,WRK,ENPH,MHK,MSFT
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-03,0.002814,0.017847,-0.024288,0.019457,0.018935,-0.003354,-0.001895,0.002099,0.008878,-0.001525,-0.021534,0.011038,0.002643,0.027816,0.002255,0.022546,0.024135,0.000174,0.011239,0.002417,-0.002909,-0.011343,0.010276,0.029806,-0.003545,0.003980,0.009058,-0.022892,-0.006682,-0.004619,0.014471,0.002405,0.012103,0.030737,0.004869,-0.005526,0.003224,0.003253,0.017283,0.008359,...,-0.002373,0.023341,-0.009511,0.020090,0.014403,-0.005601,0.008898,0.012045,0.003498,0.031853,0.002786,0.022624,0.011558,0.013479,0.016010,-0.008959,0.020070,-0.009994,0.002019,0.004875,0.009249,-0.000851,-0.003103,0.000923,0.009392,0.006113,0.021983,0.027861,-0.006127,0.002625,0.018684,0.003838,0.021743,0.034876,0.057181,0.004737,0.008864,0.039604,0.010367,0.007081
2017-01-04,-0.014031,0.012820,0.042403,0.018642,0.013240,0.014100,0.003639,0.016754,0.016637,0.001145,-0.007279,0.000631,0.000942,0.003976,0.006976,-0.000678,0.000773,0.014959,0.011415,0.000241,0.013053,0.010215,-0.003179,0.015060,0.017609,0.023125,0.015182,-0.014235,0.009146,-0.004206,0.000560,-0.005713,0.020966,-0.035927,0.008603,0.029237,0.030435,0.014358,0.001355,-0.001223,...,0.000188,-0.002553,-0.013604,0.023111,0.007795,0.001877,0.055192,0.017734,0.025823,0.033270,-0.002885,0.020970,0.009141,-0.004469,0.008788,0.014631,0.015398,0.034115,0.022163,0.013167,0.023720,0.007663,-0.001442,-0.004305,0.020937,0.046272,0.001422,0.007995,0.009145,0.014904,0.016091,0.000261,-0.000333,0.015440,0.030664,0.001925,0.015033,0.095238,0.020471,-0.004474
2017-01-05,0.000086,-0.000558,0.031950,-0.011765,-0.009170,0.007584,0.003310,-0.004119,-0.014065,0.004958,-0.012108,0.004058,-0.002258,-0.005366,0.006704,-0.003055,0.005404,0.026907,-0.078705,0.000602,-0.005609,0.013535,0.012117,0.018546,-0.004195,-0.015499,-0.000520,-0.000903,-0.003263,-0.012087,-0.002236,-0.005745,-0.003955,-0.003093,-0.002353,-0.001350,-0.021097,-0.016210,0.016740,-0.007756,...,0.019717,0.003925,-0.009735,-0.045374,0.002486,-0.010989,-0.018873,-0.019062,-0.017495,-0.001660,-0.019610,-0.000189,-0.001359,-0.006092,0.009612,-0.003165,0.001685,-0.003163,0.023325,-0.012996,-0.003160,-0.012885,-0.006767,-0.006178,-0.071144,-0.004914,-0.013248,0.001738,-0.001324,-0.009129,-0.021263,0.000521,-0.000333,-0.014941,-0.039256,-0.006435,0.004616,-0.026087,-0.007432,0.000000
2017-01-06,0.020697,0.014900,0.009097,0.000000,0.014577,0.000313,0.012097,0.002068,-0.011522,0.007590,0.019334,0.010777,0.001886,0.003018,0.010432,0.005107,0.009887,-0.001335,-0.013056,0.000843,0.018445,0.003530,0.021424,-0.005614,0.006494,-0.001640,0.004424,-0.003011,0.007092,0.006781,0.004856,-0.002543,-0.018173,-0.015071,0.011399,0.004424,0.009370,0.006498,-0.002993,0.016868,...,-0.000491,-0.015808,0.005735,-0.006173,0.010196,-0.000689,-0.010992,0.008107,-0.012426,0.014796,-0.000983,0.016020,0.004535,0.000430,-0.003868,0.018817,0.003364,0.012422,0.002247,0.003465,-0.024828,0.006634,0.005359,-0.006061,0.018330,-0.006734,-0.007288,0.013178,-0.000816,-0.007010,0.002489,0.001997,0.002662,0.011141,0.003871,0.010247,0.004978,-0.008929,0.000440,0.008668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

## Weighted returns

In [12]:
def generate_weighted_returns(returns, weights):
  """
    Generate weighted returns.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    weights : DataFrame
        Weights for each ticker and date

    Returns
    -------
    weighted_returns : DataFrame
        Weighted returns for each ticker and date
  """

  assert returns.index.equals(weights.index)
  assert returns.columns.equals(weights.columns)

  weighted_returns = returns * weights

  return weighted_returns

In [13]:
sp_weighted_returns = generate_weighted_returns(sp_price_returns, sp_weights)
sp_weighted_returns

,IDXX,DIS,DXCM,BAC,MS,ABBV,YUM,HIG,DFS,AEE,XLNX,RTX,EXPD,HSIC,BAX,MGM,EOG,O,BBWI,WBA,AIG,EQR,WMB,NFLX,ICE,TRMB,BLL,KR,SYY,TEL,LRCX,KHC,VIAC,VLO,SRE,DPZ,QRVO,ZION,BMY,TXN,...,COST,OKE,LW,RHI,INFO,IFF,GM,WAB,DLTR,CNC,MKC,NKE,BSX,CAT,PFE,WST,NWSA,NSC,PEAK,RF,PHM,TFC,CB,BKR,STZ,NCLH,SBAC,SPG,MAA,XYL,COF,UPS,MRK,CAH,GPS,HSY,WRK,ENPH,MHK,MSFT
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-03,1.656642e-06,1.415821e-04,-0.000018,0.000349,0.000053,-1.380337e-05,-4.662819e-06,3.380628e-06,1.175440e-05,-6.853116e-07,-0.000043,0.000026,2.014144e-06,3.272394e-05,0.000002,0.000052,0.000084,1.421547e-07,0.000015,7.605164e-06,-0.000011,-0.000016,1.252298e-05,0.000315,-0.000004,1.797912e-06,5.312183e-06,-0.000059,-0.000011,-4.980904e-06,2.450179e-05,4.714693e-06,2.234480e-05,0.000068,4.176753e-06,-5.888099e-06,0.000003,3.191352e-06,0.000079,0.000023,...,-6.254317e-06,1.584857e-05,-5.905007e-06,1.074637e-05,0.000013,-2.770844e-06,0.000026,8.402718e-06,0.000006,0.000033,1.093592e-06,1.309269e-04,0.000015,0.000050,0.000086,-0.000004,3.888718e-06,-0.000015,1.289512e-06,8.515147e-06,4.782359e-06,-0.000002,-0.000005,8.687199e-07,0.000024,0.000004,2.553367e-05,0.000097,-4.453620e-06,1.730513e-06,0.000046,8.063336e-06,0.000120,8.631893e-05,0.000091,0.000003,3.512345e-06,1.629908e-07,9.059824e-06,0.000075
2017-01-04,-3.101389e-04,1.015537e-04,0.000043,0.000278,0.000037,5.760542e-05,5.594364e-06,2.027506e-05,2.426871e-05,5.459021e-07,-0.000012,0.000002,6.765144e-07,2.408837e-06,0.000006,-0.000001,0.000002,1.195276e-05,0.000019,6.669774e-07,0.000035,0.000010,-3.020399e-06,0.000141,0.000027,9.109645e-06,1.153487e-05,-0.000036,0.000012,-5.407618e-06,6.683084e-07,-8.586768e-06,6.279550e-05,-0.000135,9.775573e-06,2.874335e-05,0.000023,9.905306e-06,0.000004,-0.000003,...,5.367284e-07,-1.728368e-06,-7.480828e-06,1.665510e-05,0.000004,1.301097e-06,0.000383,1.485946e-05,0.000062,0.000039,-1.546207e-06,1.349465e-04,0.000012,-0.000013,0.000049,0.000007,1.990818e-06,0.000086,2.638800e-05,1.501867e-05,2.491592e-05,0.000013,-0.000002,-3.324606e-06,0.000085,0.000036,9.660252e-07,0.000021,6.245624e-06,1.119228e-05,0.000033,4.415362e-07,-0.000002,2.254369e-05,0.000057,0.000001,7.599483e-06,9.149569e-07,2.121457e-05,-0.000051
2017-01-05,1.212899e-07,-3.344582e-06,0.000030,-0.000195,-0.000032,1.926528e-05,7.267585e-06,-2.677196e-06,-1.992892e-05,4.802691e-06,-0.000013,0.000013,-9.307045e-07,-2.242008e-06,0.000006,-0.000007,0.000015,2.775760e-05,-0.000303,2.986977e-06,-0.000012,0.000012,2.027219e-05,0.000230,-0.000003,-6.181036e-06,-3.494741e-07,-0.000002,-0.000005,-1.035366e-05,-3.352671e-06,-1.144957e-05,-9.195038e-06,-0.000010,-3.851658e-06,-9.393733e-07,-0.000014,-1.213199e-05,0.000069,-0.000020,...,7.645934e-05,2.078182e-06,-3.400363e-06,-4.667716e-05,0.000001,-1.076617e-05,-0.000086,-7.743835e-06,-0.000036,-0.000001,-2.149366e-05,-7.649663e-07,-0.000002,-0.000022,0.000050,-0.000001,4.185450e-07,-0.000009,2.629171e-05,-1.903953e-05,-2.633800e-06,-0.000017,-0.000013,-4.525378e-06,-0.000949,-0.000002,-8.706551e-06,0.000005,-9.096363e-07,-7.354351e-06,-0.000036,8.941133e-07,-0.000002,-1.675060e-05,-0.000095,-0.000009,1.790923e-06,-8.334356e-08,-7.584756e-06,0.000000
2017-01-06,2.204795e-05,1.412490e-04,0.000005,0.000000,0.000050,7.223250e-07,1.961084e-05,2.517672e-06,-2.238400e-05,6.365788e-06,0.000024,0.000027,7.940189e-07,1.857417e-06,0.000013,0.000009,0.000022,-1.154511e-06,-0.000026,2.297694e-06,0.000058,0.000003,2.959721e-05,-0.000075,0.000005,-4.508399e-07,1.565338e-06,-0.000008,0.000016,4.756030e-06,5.544931e-06,-4.196586e-06,-5.380238e-05,-0.000029,9.039735e-06,3.620453e-06,0.000005,5.140

## Cumulative returns

To compare performance between the ETF and Index, we're going to calculate the tracking error. Before we do that, we first need to calculate the index and ETF comulative returns. Implement `calculate_cumulative_returns` to calculate the cumulative returns over time given the returns.

In [14]:
def calculate_cumulative_returns(returns):
  """
    Calculate cumulative returns.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date

    Returns
    -------
    cumulative_returns : Pandas Series
        Cumulative returns for each date
  """

  cumulative_returns = (returns.sum(axis=1)+1).cumprod()

  return cumulative_returns

In [15]:
sp_weighted_cumulative_returns = calculate_cumulative_returns(sp_weighted_returns)
sp_weighted_cumulative_returns

Date
2017-01-02    1.000000
2017-01-03    1.008213
2017-01-04    1.018724
2017-01-05    1.016647
2017-01-06    1.022251
                ...   
2020-12-24    3.822257
2020-12-28    3.865893
2020-12-29    3.862944
2020-12-30    3.900251
2020-12-31    3.928875
Length: 1010, dtype: float64

## Covariance

If we have $m$ stock series, the covariance matrix is an $m \times m$ matrix containing the covariance between each pair of stocks.  We can use [`Numpy.cov`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.cov.html) to get the covariance.  We give it a 2D array in which each row is a stock series, and each column is an observation at the same period of time. For any `NaN` values, you can replace them with zeros using the [`DataFrame.fillna`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html) function.

The covariance matrix $\mathbf{P} = 
\begin{bmatrix}
\sigma^2_{1,1} & ... & \sigma^2_{1,m} \\ 
... & ... & ...\\
\sigma_{m,1} & ... & \sigma^2_{m,m}  \\
\end{bmatrix}$

In [16]:
def get_covariance_returns(returns):
  """
    Calculate covariance matrices.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date

    Returns
    -------
    returns_covariance  : 2 dimensional Ndarray
        The covariance of the returns
  """

  returns_covariance = returns.fillna(0)
  returns_covariance = np.cov(returns_covariance.T)

  return returns_covariance

In [17]:
covariance_returns = get_covariance_returns(sp_price_returns)
covariance_returns = pd.DataFrame(covariance_returns, sp_price_returns.columns, sp_price_returns.columns)
covariance_returns

,IDXX,DIS,DXCM,BAC,MS,ABBV,YUM,HIG,DFS,AEE,XLNX,RTX,EXPD,HSIC,BAX,MGM,EOG,O,BBWI,WBA,AIG,EQR,WMB,NFLX,ICE,TRMB,BLL,KR,SYY,TEL,LRCX,KHC,VIAC,VLO,SRE,DPZ,QRVO,ZION,BMY,TXN,...,COST,OKE,LW,RHI,INFO,IFF,GM,WAB,DLTR,CNC,MKC,NKE,BSX,CAT,PFE,WST,NWSA,NSC,PEAK,RF,PHM,TFC,CB,BKR,STZ,NCLH,SBAC,SPG,MAA,XYL,COF,UPS,MRK,CAH,GPS,HSY,WRK,ENPH,MHK,MSFT
IDXX,0.000380,0.000122,0.000235,0.000142,0.000177,0.000139,0.000138,0.000112,0.000217,0.000102,0.000201,0.000145,0.000132,0.000124,0.000148,0.000232,0.000115,0.000124,0.000183,0.000080,0.000148,0.000103,0.000132,0.000192,0.000132,0.000199,0.000137,0.000041,0.000118,0.000173,0.000271,0.000114,0.000163,0.000149,0.000111,0.000106,0.000227,0.000079,0.000113,0.000197,...,0.000113,0.000142,0.000099,0.000137,0.000169,0.000105,0.000144,0.000125,0.000077,0.000188,0.000115,0.000159,0.000190,0.000141,0.000107,0.000172,0.000123,0.000157,0.000133,0.000131,0.000181,0.000126,0.000111,0.000145,0.000113,0.000171,0.000138,0.000090,0.000124,0.000140,0.000173,0.000132,0.000126,0.000124,0.000161,0.000094,0.000141,0.000223,0.000160,0.000218
DIS,0.000122,0.000355,0.000133,0.000258,0.000258,0.000112,0.000162,0.000233,0.000338,0.000102,0.000164,0.000227,0.000121,0.000153,0.000098,0.000345,0.000268,0.000184,0.000255,0.000146,0.000285,0.000151,0.000224,0.000112,0.000127,0.000209,0.000107,0.000035,0.000250,0.000188,0.000235,0.000134,0.000282,0.000287,0.000139,0.000067,0.000202,0.000217,0.000092,0.000171,...,0.000091,0.000310,0.000183,0.000230,0.000169,0.000143,0.000235,0.000226,0.000125,0.000151,0.000078,0.000176,0.000173,0.000196,0.000107,0.000092,0.000193,0.000209,0.000166,0.000269,0.000199,0.000248,0.000160,0.000280,0.000166,0.000456,0.000093,0.000310,0.000142,0.000181,0.000287,0.000128,0.000093,0.000165,0.000288,0.000103,0.000245,0.000239,0.000226,0.000170
DXCM,0.000235,0.000133,0.001080,0.000144,0.000158,0.000115,0.000139,0.000148,0.000202,0.000079,0.000176,0.000141,0.000120,0.000092,0.000119,0.000262,0.000155,0.000139,0.000221,0.000079,0.000161,0.000098,0.000140,0.000222,0.000141,0.000269,0.000119,0.000051,0.000147,0.000162,0.000217,0.000090,0.000170,0.000143,0.000115,0.000133,0.000223,0.000104,0.000096,0.000165,...,0.000102,0.000175,0.000111,0.000125,0.000163,0.000075,0.000146,0.000137,0.000101,0.000228,0.000097,0.000171,0.000211,0.000119,0.000073,0.000178,0.000118,0.000158,0.000117,0.000167,0.000174,0.000146,0.000092,0.000125,0.000103,0.000203,0.000113,0.000119,0.000102,0.000119,0.000180,0.000130,0.000100,0.000111,0.000178,0.000097,0.000100,0.000331,0.000105,0.000208
BAC,0.000142,0.000258,0.000144,0.000499,0.000437,0.000162,0.000194,0.000330,0.000506,0.000133,0.000226,0.000307,0.000182,0.000219,0.000162,0.000451,0.000378,0.000221,0.000389,0.000229,0.000401,0.000208,0.000294,0.000139,0.000183,0.000268,0.000154,0.000069,0.000301,0.000279,0.000339,0.000195,0.000322,0.000412,0.000175,0.000073,0.000284,0.000422,0.000131,0.000242,...,0.000108,0.000400,0.000240,0.000317,0.000226,0.000209,0.000351,0.000327,0.000158,0.000234,0.000107,0.000220,0.000230,0.000305,0.000153,0.000124,0.000259,0.000299,0.000223,0.000503,0.000243,0.000460,0.000260,0.000386,0.000214,0.000562,0.000122,0.000384,0.000181,0.000277,0.000479,0.000191,0.000132,0.000229,0.000439,0.000120,0.000385,0.000285,0.000319,0.000219
MS,0.000177,0.000258,0.000158,0.000437,0.000498,0.000174,0.000200,0.000304,0.000496,0.000137,0.000250,0.000304,0.000198,0.000209,0.000160,0.000465,0.000358,0.000215,0.000386,0.000218,0.000391,0.000185,0.000301,0.000161,0.000200,0.000284,0.000159,0.000063,0.000290,0.000289,0.000371,0.000215,0.000328,0.000389,0.000172,0.000076,0.000314,0.000371,0.000144,0.000260,...,0.000123,0.000382,0.000231,0.000296,0.000240,0.000207,0.000338,0.000312,0.000183,0.000240,0.000121,0.000231,0.000228,0.000307,0.000154,0.000134,0.000263,0.000311,0.000220,0.000455,0.000273,0.000414,0.000250,0.000385,0.000212,0.000544,0.000138,0.000342,0.000177,0.000283,0.000464,0.000194,0.000139,0.000230,0.000428,0.000135,0.000378,0.000325,0.000326,0.000246
...,...,...,...,...,

## portfolio variance
We can write the portfolio variance $\sigma^2_p = \mathbf{x^T} \mathbf{P} \mathbf{x}$

The $\mathbf{x^T} \mathbf{P} \mathbf{x}$ is called the quadratic form.
We can use the cvxpy function `quad_form(x,P)` to get the quadratic form.

## Distance from index weights
We want portfolio weights that track the index closely.  So we want to minimize the distance between them.
You can get the distance between two points in an x,y plane by adding the square of the x and y distances and taking the square root.  Extending this to any number of dimensions is called the L2 norm.  So: $\sqrt{\sum_{1}^{n}(weight_i - indexWeight_i)^2}$  Can also be written as $\left \| \mathbf{x} - \mathbf{index} \right \|_2$.  There's a cvxpy function called [norm()](https://www.cvxpy.org/api_reference/cvxpy.atoms.other_atoms.html#norm)
`norm(x, p=2, axis=None)`.  The default is already set to find 'an L2 norm, so you would pass in one argument, which is the difference between your portfolio weights and the index weights.

## objective function
We want to minimize both the portfolio variance and the distance of the portfolio weights from the index weights.
We also want to choose a `scale` constant, which is $\lambda$ in the expression. 

$\mathbf{x^T} \mathbf{P} \mathbf{x} + \lambda \left \| \mathbf{x} - \mathbf{index} \right \|_2$


This lets us choose how much priority we give to minimizing the difference from the index, relative to minimizing the variance of the portfolio by choosing a higher value for `scale` ($\lambda$).

We can find the objective function using cvxpy `objective = cvx.Minimize()`.



## constraints
We can also define our constraints in a list.  For example, you'd want the weights to sum to one. So $\sum_{1}^{n}x = 1$.  You may also need to go long only, which means no shorting, so no negative weights.  So $x_i >0 $ for all $i$. you could save a variable as `[x >= 0, sum(x) == 1]`, where x was created using `cvx.Variable()`.

## optimization
So now that we have our objective function and constraints, we can solve for the values of $\mathbf{x}$.
cvxpy has the constructor `Problem(objective, constraints)`, which returns a `Problem` object.

The `Problem` object has a function solve(), which returns the minimum of the solution.  In this case, this is the minimum variance of the portfolio.

It also updates the vector $\mathbf{x}$.

We can check out the values of $x_A$ and $x_B$ that gave the minimum portfolio variance by using `x.value`

In [18]:
def get_optimal_weights(covariance_returns, index_weights, scale=2.0):
  """
    Find the optimal weights.

    Parameters
    ----------
    covariance_returns : 2 dimensional Ndarray
        The covariance of the returns
    index_weights : Pandas Series
        Index weights for all tickers at a period in time
    scale : int
        The penalty factor for weights the deviate from the index 
    Returns
    -------
    x : 1 dimensional Ndarray
        The solution for x
  """

  assert len(covariance_returns.shape) == 2
  assert len(index_weights.shape) == 1
  assert covariance_returns.shape[0] == covariance_returns.shape[1] == index_weights.shape[0]
  
  # number of stocks m is number of rows of returns, and also number of index weights
  m = len(index_weights) 
  # covariance matrix of returns
  P = covariance_returns
  # x variables (to be found with optimisation)
  x = cvx.Variable(m)
  # portfolio variance in quadratic form
  portfolio_variance = cvx.quad_form(x, P)
  # euclidean distance (L2 norm) between portfolio and index weights
  distance_to_index = cvx.norm(x - index_weights)
  # objective function
  objective = cvx.Minimize(portfolio_variance + scale*distance_to_index)
  # constraints
  constraints = [x>=0, sum(x)==1]
  # solve objective function
  problem = cvx.Problem(objective, constraints)
  result = problem.solve()
  # get weights of optimized portfolio
  x_values = x.value

  return x_values

## Optimised portfolio without rebalancing

Using the `get_optimal_weights` function, let's generate the optimal ETF weights without rebalanceing. We can do this by feeding in the covariance of the entire history of data. We also need to feed in a set of index weights. We'll go with the average weights of the index over time.

In [19]:
raw_optimal_single_rebalance_etf_weights = get_optimal_weights(covariance_returns.values, sp_weights.iloc[-1])
print(raw_optimal_single_rebalance_etf_weights)
optimal_single_rebalance_etf_weights = pd.DataFrame(
    np.tile(raw_optimal_single_rebalance_etf_weights, (len(sp_price_returns.index), 1)),
    sp_price_returns.index,
    sp_price_returns.columns)
optimal_single_rebalance_etf_weights

[6.47598687e-04 7.26936519e-03 1.69489959e-03 4.89199153e-03
 2.02512133e-03 3.60887373e-03 1.00802399e-03 4.87426359e-04
 6.23716532e-04 5.40921061e-04 9.94009203e-04 1.67383324e-03
 2.51791969e-04 1.52116015e-04 8.00415844e-04 2.80881239e-03
 6.61167731e-04 7.87467385e-04 5.29455691e-04 1.70973044e-03
 6.26771047e-04 4.70869871e-04 7.54627757e-04 1.65388892e-02
 9.58579848e-04 2.64811578e-04 5.49728430e-04 1.25784236e-03
 4.81596673e-04 5.18028414e-04 4.43519060e-03 6.46730630e-04
 1.60731555e-03 7.80677670e-04 6.73825646e-04 9.21580664e-04
 7.86749924e-04 1.77946206e-04 2.99753232e-03 2.65970902e-03
 1.65597826e-03 1.79700251e-04 6.56875172e-04 9.03137282e-04
 4.20364055e-04 2.27748670e-04 5.21874244e-04 3.77171465e-04
 2.46051519e-03 5.69379374e-04 1.74693977e-03 1.29688962e-02
 3.26592868e-04 2.46744569e-04 2.07591963e-03 2.13806394e-03
 1.22281552e-03 1.94539751e-03 5.22049856e-04 3.74775183e-04
 2.60918428e-04 2.35219232e-04 6.09949102e-04 3.81865764e-04
 8.19144456e-04 2.371921

,IDXX,DIS,DXCM,BAC,MS,ABBV,YUM,HIG,DFS,AEE,XLNX,RTX,EXPD,HSIC,BAX,MGM,EOG,O,BBWI,WBA,AIG,EQR,WMB,NFLX,ICE,TRMB,BLL,KR,SYY,TEL,LRCX,KHC,VIAC,VLO,SRE,DPZ,QRVO,ZION,BMY,TXN,...,COST,OKE,LW,RHI,INFO,IFF,GM,WAB,DLTR,CNC,MKC,NKE,BSX,CAT,PFE,WST,NWSA,NSC,PEAK,RF,PHM,TFC,CB,BKR,STZ,NCLH,SBAC,SPG,MAA,XYL,COF,UPS,MRK,CAH,GPS,HSY,WRK,ENPH,MHK,MSFT
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-01-02,0.000648,0.007269,0.001695,0.004892,0.002025,0.003609,0.001008,0.000487,0.000624,0.000541,0.000994,0.001674,0.000252,0.000152,0.0008,0.002809,0.000661,0.000787,0.000529,0.00171,0.000627,0.000471,0.000755,0.016539,0.000959,0.000265,0.00055,0.001258,0.000482,0.000518,0.004435,0.000647,0.001607,0.000781,0.000674,0.000922,0.000787,0.000178,0.002998,0.00266,...,0.003457,0.00045,0.000169,0.000186,0.000847,0.002312,0.002621,0.000296,0.000687,0.000634,0.000641,0.002356,0.001267,0.001747,0.006234,0.000731,0.000185,0.000912,0.000293,0.00043,0.000423,0.000945,0.001056,0.000323,0.000857,0.002105,0.00113,0.001177,0.00027,0.00029,0.000717,0.002219,0.002288,0.000374,0.000392,0.00041,0.000334,0.005595,0.000291,0.02628
2017-01-03,0.000648,0.007269,0.001695,0.004892,0.002025,0.003609,0.001008,0.000487,0.000624,0.000541,0.000994,0.001674,0.000252,0.000152,0.0008,0.002809,0.000661,0.000787,0.000529,0.00171,0.000627,0.000471,0.000755,0.016539,0.000959,0.000265,0.00055,0.001258,0.000482,0.000518,0.004435,0.000647,0.001607,0.000781,0.000674,0.000922,0.000787,0.000178,0.002998,0.00266,...,0.003457,0.00045,0.000169,0.000186,0.000847,0.002312,0.002621,0.000296,0.000687,0.000634,0.000641,0.002356,0.001267,0.001747,0.006234,0.000731,0.000185,0.000912,0.000293,0.00043,0.000423,0.000945,0.001056,0.000323,0.000857,0.002105,0.00113,0.001177,0.00027,0.00029,0.000717,0.002219,0.002288,0.000374,0.000392,0.00041,0.000334,0.005595,0.000291,0.02628
2017-01-04,0.000648,0.007269,0.001695,0.004892,0.002025,0.003609,0.001008,0.000487,0.000624,0.000541,0.000994,0.001674,0.000252,0.000152,0.0008,0.002809,0.000661,0.000787,0.000529,0.00171,0.000627,0.000471,0.000755,0.016539,0.000959,0.000265,0.00055,0.001258,0.000482,0.000518,0.004435,0.000647,0.001607,0.000781,0.000674,0.000922,0.000787,0.000178,0.002998,0.00266,...,0.003457,0.00045,0.000169,0.000186,0.000847,0.002312,0.002621,0.000296,0.000687,0.000634,0.000641,0.002356,0.001267,0.001747,0.006234,0.000731,0.000185,0.000912,0.000293,0.00043,0.000423,0.000945,0.001056,0.000323,0.000857,0.002105,0.00113,0.001177,0.00027,0.00029,0.000717,0.002219,0.002288,0.000374,0.000392,0.00041,0.000334,0.005595,0.000291,0.02628
2017-01-05,0.000648,0.007269,0.001695,0.004892,0.002025,0.003609,0.001008,0.000487,0.000624,0.000541,0.000994,0.001674,0.000252,0.000152,0.0008,0.002809,0.000661,0.000787,0.000529,0.00171,0.000627,0.000471,0.000755,0.016539,0.000959,0.000265,0.00055,0.001258,0.000482,0.000518,0.004435,0.000647,0.001607,0.000781,0.000674,0.000922,0.000787,0.000178,0.002998,0.00266,...,0.003457,0.00045,0.000169,0.000186,0.000847,0.002312,0.002621,0.000296,0.000687,0.000634,0.000641,0.002356,0.001267,0.001747,0.006234,0.000731,0.000185,0.000912,0.000293,0.00043,0.000423,0.000945,0.001056,0.000323,0.000857,0.002105,0.00113,0.001177,0.00027,0.00029,0.000717,0.002219,0.002288,0.000374,0.000392,0.00041,0.000334,0.005595,0.000291,0.02628
2017-01-06,0.000648,0.007269,0.001695,0.004892,0.002025,0.003609,0.001008,0.000487,0.000624,0.000541,0.000994,0.001674,0.000252,0.000152,0.0008,0.002809,0.000661,0.000787,0.000529,0.00171,0.000627,0.000471,0.000755,0.016539,0.000959,0.000265,0.00055,0.001258,0.000482,0.000518,0.004435,0.000647,0.001607,0.000781,0.000674,0.000922,0.000787,0.000178,0.002998,0.00266,...,0.003457,0.00045,0.000169,0.000186,0.000847,0.002312,0.002621,0.000296,0.000687,0.000634,0.000641,0.002356,0.001267,0.001747,0.006234,0.000731,0.000185,0.000912,0.000293,0.00043,0.000423,0.000945,0.001056,0.000323,0.000857,0.002105,0.00113,0.001177,0.00027,0.00029,0.000717,0.002219,0.002288,0.000374,0.000392,0.0

With our ETF weights built, let's compare it to the index. Run the next cell to calculate the ETF returns and compare it to the index returns.

In [20]:
optim_etf_returns = generate_weighted_returns(sp_price_returns, optimal_single_rebalance_etf_weights)
optim_etf_cumulative_returns = calculate_cumulative_returns(optim_etf_returns)

## Tracking error

In order to check the performance of the smart beta portfolio, we can calculate the annualized tracking error against the index. Implement `tracking_error` to return the tracking error between the ETF and benchmark.

For reference, we'll be using the following annualized tracking error function:
$$ TE = \sqrt{252} * SampleStdev(r_p - r_b) $$

Where $ r_p $ is the portfolio/ETF returns and $ r_b $ is the benchmark returns.

_Note: When calculating the sample standard deviation, the delta degrees of freedom is 1, which is the also the default value._

In [21]:
def tracking_error(benchmark_returns_by_date, etf_returns_by_date):
    """
    Calculate the tracking error.

    Parameters
    ----------
    benchmark_returns_by_date : Pandas Series
        The benchmark returns for each date
    etf_returns_by_date : Pandas Series
        The ETF returns for each date

    Returns
    -------
    tracking_error : float
        The tracking error
    """
    assert benchmark_returns_by_date.index.equals(etf_returns_by_date.index)
    
    #TODO: Implement function
    tracking_error = (etf_returns_by_date - benchmark_returns_by_date).std()*(252**0.5)

    return tracking_error

In [22]:
optim_etf_tracking_error = tracking_error(np.sum(sp_weighted_returns, 1), np.sum(optim_etf_returns, 1))
print('Optimized ETF Tracking Error: {}'.format(optim_etf_tracking_error))

Optimized ETF Tracking Error: 0.09575339648086863


## Rebalance portfolio over time

The single optimized ETF portfolio used the same weights for the entire history. This might not be the optimal weights for the entire period. Let's rebalance the portfolio over the same period instead of using the same weights. Implement `rebalance_portfolio` to rebalance a portfolio.

Reblance the portfolio every n number of days, which is given as `shift_size`. When rebalancing, you should look back a certain number of days of data in the past, denoted as `chunk_size`. Using this data, compute the optoimal weights using `get_optimal_weights` and `get_covariance_returns`.

In [23]:
def rebalance_portfolio(returns, index_weights, shift_size, chunk_size):
    """
    Get weights for each rebalancing of the portfolio.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    index_weights : DataFrame
        Index weight for each ticker and date
    shift_size : int
        The number of days between each rebalance
    chunk_size : int
        The number of days to look in the past for rebalancing

    Returns
    -------
    all_rebalance_weights  : list of Ndarrays
        The ETF weights for each point they are rebalanced
    """
    assert returns.index.equals(index_weights.index)
    assert returns.columns.equals(index_weights.columns)
    assert shift_size > 0
    assert chunk_size >= 0
    
    #TODO: Implement function
    all_rebalance_weights = []
    
    for i in range(chunk_size, len(returns), shift_size):
        chunks = returns.iloc[i - chunk_size : i]
        cov_returns = get_covariance_returns(chunks)
        optimal_weights = get_optimal_weights(cov_returns, index_weights.iloc[i-1])
        all_rebalance_weights.append(optimal_weights)
        
    return all_rebalance_weights

In [24]:
chunk_size = 250
shift_size = 5
all_rebalance_weights = rebalance_portfolio(sp_price_returns, sp_weights, shift_size, chunk_size)
all_rebalance_weights

[array([6.22425379e-04, 7.37249696e-03, 4.45913548e-04, 1.21115536e-02,
        3.45794224e-03, 1.97368869e-03, 1.02027076e-03, 5.54295223e-04,
        1.87381496e-03, 7.22817483e-04, 1.26888428e-03, 2.65759233e-03,
        4.04001219e-04, 8.29697175e-04, 1.44595007e-03, 1.98785009e-03,
        2.26169320e-03, 7.63862582e-04, 1.33743046e-03, 2.64309645e-03,
        1.90495534e-03, 1.14545054e-03, 1.22641230e-03, 9.33392929e-03,
        2.53021916e-03, 1.87150747e-04, 8.86742276e-04, 1.57678877e-03,
        8.14365681e-04, 1.24605017e-03, 3.50789514e-03, 1.44906135e-03,
        2.54856187e-03, 2.30344775e-03, 1.28163689e-03, 1.02076625e-03,
        9.56346202e-04, 3.97658349e-04, 2.56589336e-03, 2.10440436e-03,
        2.15318948e-03, 7.36646200e-04, 1.25853416e-03, 1.05980218e-03,
        4.35602945e-04, 5.27365786e-04, 1.63881888e-03, 5.33617127e-04,
        1.96918671e-03, 8.50265151e-04, 2.77823029e-03, 3.02260955e-03,
        5.50243123e-04, 4.62238285e-04, 1.37203791e-03, 3.204982

## Portfolio turnover

With the portfolio rebalanced, we need to use a metric to measure the cost of rebalancing the portfolio. Implement `get_portfolio_turnover` to calculate the annual portfolio turnover.

$ AnnualizedTurnover =\frac{SumTotalTurnover}{NumberOfRebalanceEvents} * NumberofRebalanceEventsPerYear $

$ SumTotalTurnover =\sum_{t,n}{\left | x_{t,n} - x_{t+1,n} \right |} $ Where $ x_{t,n} $ are the weights at time $ t $ for equity $ n $.

$ SumTotalTurnover $ is just a different way of writing $ \sum \left | x_{t_1,n} - x_{t_2,n} \right | $

In [25]:
def get_portfolio_turnover(all_rebalance_weights, shift_size, rebalance_count, n_trading_days_in_year=252):
    """
    Calculage portfolio turnover.

    Parameters
    ----------
    all_rebalance_weights : list of Ndarrays
        The ETF weights for each point they are rebalanced
    shift_size : int
        The number of days between each rebalance
    rebalance_count : int
        Number of times the portfolio was rebalanced
    n_trading_days_in_year: int
        Number of trading days in a year

    Returns
    -------
    portfolio_turnover  : float
        The portfolio turnover
    """
    assert shift_size > 0
    assert rebalance_count > 0
    
    #TODO: Implement function
    sum_total_turnover = np.sum(np.abs(np.diff(np.flip(all_rebalance_weights, axis=0), axis=0)))
    no_rebalance_events_per_year = n_trading_days_in_year / shift_size
    portfolio_turnover = (sum_total_turnover / rebalance_count) * no_rebalance_events_per_year
    
    return portfolio_turnover

In [26]:
print(get_portfolio_turnover(all_rebalance_weights, shift_size, len(all_rebalance_weights) - 1))

17.417027014477796
